In [1]:
import requests
import bs4
import pandas as pd
import json
import time
import random
import urllib.parse
import math
from tqdm import tqdm
from datetime import date

In [ ]:
# 有問題：ＩＰ使用方式應該不對

In [2]:
path = 'url_data/'
user_agent = [
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
    "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
    "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
    "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
    "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
    "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
    "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20"
]

In [3]:
N_IPs = 30
import asyncio
from proxybroker import Broker
list_proxy = []
async def view_proxy(proxies_queue): 
    while True: 
        proxy = await proxies_queue.get() 
        p = str(proxy).split('] ')[1]
        p = p.split(">")[0]
        list_proxy.append(p)
        if proxy is None:
            break 
        print(proxy)

proxies_queue = asyncio.Queue()
broker = Broker(proxies_queue)
# tasks = asyncio.gather(broker.grab(limit=300), view_proxy(proxies_queue))
tasks = asyncio.gather(broker.find(types=['HTTP', 'HTTPS'], limit=N_IPs), view_proxy(proxies_queue))
loop = asyncio.get_event_loop()

# functions

In [4]:
def get_comment_dict(comment_container):
    try:
        dict_comment = dict()
        dict_comment['last_collected_date '] = str(date.today())
        dict_comment['author_id '] = comment_container.get('data-author-id')
        dict_comment['comment_id '] = comment_container.get('data-comment-id')
        dict_comment['comment'] = comment_container.find(attrs={"class": "comment-body"}).text.split('\n')[0]
        return dict_comment
    except Exception as e:
        print('comment_container = ', comment_container)
        print(e)
        
def request_session(url, user_agent, ip):
    rs = requests.session()
    try:
        ua = random.choice(user_agent)
        html = rs.get(url, headers = {'User-Agent':ua, 
                                      'Connection':'close'},
                     proxies=ip)
        return html
    except Exception as e:
        print(e)
        
def get_first_10_data(html):
    obj = bs4.BeautifulSoup(html.text, 'lxml')
    comment_containers = (obj.find_all(attrs={"class": "comment-container"}))
    li_dict_comment = [get_comment_dict(com) for com in comment_containers]
    
    dict_all_comm = dict()
    dict_all_comm['web_url'] = surfing_url
    commenturl_id = str(obj.find(attrs={"class": "col-auto"})).split('"><div')[1]
    commenturl_id = commenturl_id.split('="')[1]
    dict_all_comm['commenturl_id'] = commenturl_id
    dict_all_comm['n_comments'] = int((obj.find(attrs={"class": "h4"}).text.split(' ')[0]).replace(',',''))
    dict_all_comm['comments_info'] = li_dict_comment
    return dict_all_comm

def get_first_page(surfing_url, user_agent, proxy):
    str_surfing_url = urllib.parse.quote_plus(surfing_url)
    #first page: top 10 comments
    url_base = 'https://dissenter.com/discussion/begin-extension?url='
    url = url_base+str_surfing_url 
    ip = random.choice(proxy)
    html = request_session(url, user_agent, ip)
    dict_all_comm = get_first_10_data(html)
    return dict_all_comm, ip

def get_rest_pages(dict_all_comm, user_agent, ip):
    surfing_url = dict_all_comm['web_url']
    commenturl_id = dict_all_comm['commenturl_id']
    #start from page2: load more comments
    n_page = math.ceil(dict_all_comm['n_comments']/10)
    for page_i in tqdm(range(2,n_page+1)):
        url_base_2 = 'https://dissenter.com/comment?url='
        url2_add = surfing_url+'&v=begin&uid='+commenturl_id+'&s=top&p='+str(page_i)+'&cpp=10&more=1'
        url2 = url_base_2+url2_add

        time.sleep(random.randint(4,10))
        try:
            html2 = request_session(url2, user_agent, ip)
            obj2 = bs4.BeautifulSoup(html2.text, 'lxml')
            comment_containers = (obj2.find_all(attrs={"class": "comment-container"}))
            li_dict_comment = [get_comment_dict(com) for com in comment_containers]
            dict_all_comm['comments_info'].extend(li_dict_comment)
        except Exception as e:
            print('page_i = ', page_i)
            print(e)
        if len(obj2.find_all('div')) == 1:
            print('Completed: comment data ends from page:', page_i)
            return dict_all_comm
            break
    return dict_all_comm

def write_json(dict_all_comm, surfing_url, path):
    file_name = surfing_url.split('://')[1]
    file_name = file_name.split('.com/')[0]
    file_name = file_name.replace('.','_')
    file_name = file_name.replace('/','_')
    with open(path+file_name+'.json', 'w') as f:    
        json.dump(dict_all_comm, f)
        
def get_file_one_url(surfing_url, user_agent, path, proxy):
    surfing_url = surfing_url.split('?')[0]
    dict_all_comm, ip = get_first_page(surfing_url, user_agent, proxy)
    dict_all_comm = get_rest_pages(dict_all_comm, user_agent, ip)
    try:
        file_name = surfing_url.split('://')[1]
        file_name = file_name.split('.com/')[0]
        file_name = file_name.replace('.','_')
        file_name = file_name.replace('/','_')
        with open(path+file_name+'.json') as f:
            old_file = json.load(f)
            if type(old_file) == dict:
                all_res = []
                all_res.append(old_file)
                all_res.append(dict_all_comm)
                write_json(all_res, surfing_url, path)
            else:
                dict_all_comm.append(dict_all_comm)
                write_json(dict_all_comm, surfing_url, path)
    except:
        write_json(dict_all_comm, surfing_url, path)

# main

In [5]:
# input
print('We provide two ways to use this url tracker, \
      do you what to (A) input them one by one, or (B) have an url list in json file format?\
      please input A or B:')
way = input()
if way in (['A','a']):
    print('Please input the URL you want to trace:  (ex. https://www.facebook.com/)')
    surfing_url = input()
    get_file_one_url(surfing_url, user_agent, path, proxy)
elif way in(['B','b']):
    print('please make sure your file is put under the path setted above, \
    and input your file name(without ".json"):')
    f_name = input()
    with open(path+f_name+'.json') as f:    
        url_targets = json.load(f)
        [get_file_one_url(url, user_agent, path, proxy) for url in url_targets]
else:
    print('please input A or B....')

We provide two ways to use this url tracker,       do you what to (A) input them one by one, or (B) have an url list in json file format?      please input A or B:
a
Please input the URL you want to trace:  (ex. https://www.facebook.com/)
https://en.wikipedia.org/wiki/United_States


IndexError: Cannot choose from an empty sequence

<Proxy US 0.24s [HTTP: High] 47.242.133.230:8083>
<Proxy US 0.30s [HTTP: Anonymous, HTTPS] 20.195.17.90:3128>
<Proxy US 0.37s [HTTP: Anonymous, HTTPS] 159.89.221.73:3128>
<Proxy MY 0.37s [HTTP: High, HTTPS] 219.92.3.149:8080>
<Proxy US 0.41s [HTTP: Anonymous, HTTPS] 34.94.40.85:80>
<Proxy ID 0.65s [HTTP: High] 118.99.100.131:8080>
<Proxy US 0.25s [HTTP: High] 8.210.175.82:8083>
<Proxy SG 0.26s [HTTP: High] 128.199.202.122:3128>
<Proxy US 0.29s [HTTP: High] 191.96.42.80:8080>
<Proxy UZ 0.75s [HTTP: High] 213.230.97.10:3128>
<Proxy CA 0.30s [HTTP: High] 159.203.61.169:8080>
<Proxy UZ 0.77s [HTTP: High] 84.54.82.234:3128>
<Proxy US 0.33s [HTTP: High] 198.144.166.51:80>
<Proxy JP 0.23s [HTTPS] 161.202.226.194:80>
<Proxy IN 0.39s [HTTP: High] 139.59.1.14:8080>
<Proxy CA 0.56s [HTTP: High, HTTPS] 144.217.101.245:3129>
<Proxy DE 0.39s [HTTP: High] 192.109.165.136:80>
<Proxy SG 0.39s [HTTP: High] 191.101.39.40:80>
<Proxy DE 0.39s [HTTP: High] 192.109.165.50:80>
<Proxy SG 0.39s [HTTP: High] 191